In [1]:
import sys
sys.path.append("..")

In [3]:
import time

import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

from contextlib import nullcontext

from utils import open_pickle

from cube3_game import Cube3Game
from models import Pilgrim
from g_datasets import get_torch_scrambles_3
from utils import set_seed
from utils import save_pickle
from utils import TimeContext
from utils import int_to_human

In [4]:
device = "cpu"
model_path = "../assets/models/Cube3ResnetModel_value_policy_3_8B_14M.pt"

model = Pilgrim(
    input_dim = 54, 
    hidden_dim1 = 5000, 
    hidden_dim2 = 1000, 
    num_residual_blocks = 4 
) # ~14M
model.load_state_dict(torch.load(model_path, map_location=device))    
model.to(device)

Pilgrim(
  (input_layer): Linear(in_features=324, out_features=5000, bias=True)
  (hidden_layer): Linear(in_features=5000, out_features=1000, bias=True)
  (residual_blocks): ModuleList(
    (0-3): 4 x ResidualBlock(
      (fc1): Linear(in_features=1000, out_features=1000, bias=True)
      (bn1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
      (fc2): Linear(in_features=1000, out_features=1000, bias=True)
      (bn2): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (output_layer): Linear(in_features=1000, out_features=1, bias=True)
  (output_probs_layer): Linear(in_features=1000, out_features=12, bias=True)
  (relu): ReLU()
  (bn1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.1, inplac

In [5]:
model_int8 = torch.ao.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8 # the target dtype for quantized weights
)


RuntimeError: Didn't find engine for operation quantized::linear_prepack NoQEngine